<a href="https://colab.research.google.com/github/HAZEMHOSS/Software-design-architecture/blob/main/Stadium_Tickets_Booking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Observer Pattern ---
class TicketObserver:
    def update(self, ticket):
        pass

class EmailNotifier(TicketObserver):
    def update(self, ticket):
        print(f"[Email Notification] Ticket booked for {ticket['person_name']} ({ticket['email']}) - {ticket['num_tickets']}x {ticket['ticket_type']}.")

class SMSNotifier(TicketObserver):
    def update(self, ticket):
        print(f"[SMS Notification] Ticket for {ticket['person_name']} ({ticket['phone_number']}) confirmed. Enjoy the game!")


# --- Strategy Pattern ---
class PaymentStrategy:
    def pay(self, amount):
        pass

class VodafoneCashPayment(PaymentStrategy):
    def pay(self, amount):
        print(f"[Vodafone Cash] Payment of {amount:.2f} EGP completed using Vodafone Cash.")

class CreditCardPayment(PaymentStrategy):
    def pay(self, amount):
        print(f"[Credit Card] Payment of {amount:.2f} EGP completed using Credit Card.")


# --- Decorator Pattern ---
class PaymentDecorator(PaymentStrategy):
    def __init__(self, payment_strategy):
        self.payment_strategy = payment_strategy
        self.discount_id = None

    def pay(self, amount):
        self.payment_strategy.pay(amount)

class GroupDiscountDecorator(PaymentDecorator):
    def __init__(self, payment_strategy):
        super().__init__(payment_strategy)
        self.discount_id = "Group Discount ID"

    def pay(self, amount):
        discounted_amount = amount * 0.85
        print(f"[Discount] Group Discount Applied! 15% off. New amount: {discounted_amount:.2f} EGP.")
        print(f"Discount ID: {self.discount_id}")
        self.payment_strategy.pay(discounted_amount)

class StudentDiscountDecorator(PaymentDecorator):
    def __init__(self, payment_strategy):
        super().__init__(payment_strategy)
        self.discount_id = "Student Discount ID"

    def pay(self, amount):
        discounted_amount = amount * 0.9
        print(f"[Discount] Student Discount Applied! 10% off. New amount: {discounted_amount:.2f} EGP.")
        print(f"Discount ID: {self.discount_id}")
        self.payment_strategy.pay(discounted_amount)


# --- Singleton Manager ---
class TicketManager:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance.tickets = []
            cls._instance.observers = []
        return cls._instance

    def add_observer(self, observer):
        self.observers.append(observer)

    def notify_observers(self, ticket):
        for observer in self.observers:
            observer.update(ticket)

    def add_ticket(self, person_name, email, phone_number, ticket_type, team, stadium_level, payment_strategy, num_tickets):
        ticket = {
            'person_name': person_name,
            'email': email,
            'phone_number': phone_number,
            'ticket_type': ticket_type,
            'team': team,
            'stadium_level': stadium_level,
            'payment_strategy': payment_strategy,
            'num_tickets': num_tickets
        }
        self.tickets.append(ticket)
        print(f"\nTicket successfully added for {person_name}.")
        amount = num_tickets * 100
        payment_strategy.pay(amount)
        self.notify_observers(ticket)

    def display_tickets(self):
        print("\n--- All Tickets ---")
        for ticket in self.tickets:
            print(f"Person Name: {ticket['person_name']}")
            print(f"Email: {ticket['email']}")
            print(f"Phone Number: {ticket['phone_number']}")
            print(f"Ticket Type: {ticket['ticket_type']}")
            print(f"Team: {ticket['team']}")
            print(f"Stadium Level: {ticket['stadium_level']}")
            payment_method = type(ticket['payment_strategy']).__name__.replace('Payment', '').replace('Decorator', '')
            print(f"Payment Method: {payment_method}")
            print(f"Number of Tickets: {ticket['num_tickets']}")
            print("-----------------------------")


# --- Main Application ---
def main():
    manager = TicketManager()
    manager.add_observer(EmailNotifier())
    manager.add_observer(SMSNotifier())

    ticket_types = ["Standard", "VIP"]
    teams = ["Al-Ahly", "Zamalek"]
    stadium_levels = ["Level 1", "Level 2", "Level 3"]
    payment_types = ["Vodafone Cash", "Credit Card"]

    payment_strategy_map = {
        "Vodafone Cash": VodafoneCashPayment(),
        "Credit Card": CreditCardPayment()
    }

    def get_choice(options, prompt):
        while True:
            print(f"\nSelect {prompt}:")
            for idx, option in enumerate(options, 1):
                print(f"{idx}. {option}")
            try:
                choice = int(input("Enter choice number: ").strip())
                if 1 <= choice <= len(options):
                    return options[choice - 1]
                else:
                    print("Invalid choice! Please select a valid option.")
            except ValueError:
                print("Invalid input! Please enter a number.")

    while True:
        # --- Name Validation ---
        while True:
            person_name = input("Enter your full name (first and last): ").strip()
            if len(person_name.split()) < 2:
                print("Please enter both your first and last name.")
                continue
            if all(part.replace('-', '').isalpha() for part in person_name.split()):
                break
            else:
                print("Name must contain letters only. No numbers or special characters.")

        # --- Email Validation ---
        while True:
            email = input("Enter your email (must end with @gmail.com): ").strip()
            if email.endswith("@gmail.com"):
                break
            else:
                print("Invalid email. Please enter a Gmail address ending with @gmail.com.")

        # --- Phone Number Validation ---
        while True:
            phone_number = input("Enter your phone number (numbers only, with country code): ").strip()
            if phone_number.isdigit():
                break
            else:
                print("Invalid phone number. Numbers only!")

        ticket_type = get_choice(ticket_types, "Ticket Type")
        team = get_choice(teams, "Team")
        stadium_level = get_choice(stadium_levels, "Stadium Level")
        payment_type = get_choice(payment_types, "Payment Type")
        payment_strategy = payment_strategy_map[payment_type]

        while True:
            try:
                num_tickets = int(input("\nEnter the number of tickets: ").strip())
                if num_tickets > 0:
                    break
                else:
                    print("Number of tickets must be greater than 0.")
            except ValueError:
                print("Invalid input! Please enter a number.")

        # --- Discounts ---
        print("\n--- Available Discounts ---")
        student_eligible = False
        if num_tickets >= 5:
            print("1. Group Discount (15% off) for 5 or more tickets.")
        student_input = input("Do you have a student ID? (yes/no): ").strip().lower()
        if student_input == "yes":
            student_eligible = True
            print("2. Student Discount (10% off).")

        if num_tickets >= 5:
            payment_strategy = GroupDiscountDecorator(payment_strategy)
        if student_eligible:
            payment_strategy = StudentDiscountDecorator(payment_strategy)

        # --- Add Ticket ---
        manager.add_ticket(person_name, email, phone_number, ticket_type, team, stadium_level, payment_strategy, num_tickets)

        more_tickets = input("\nDo you need another ticket? (yes/no): ").strip().lower()
        if more_tickets == "no":
            break

    manager.display_tickets()


if __name__ == "__main__":
    main()


Invalid email. Please enter a Gmail address ending with @gmail.com.
